In [1]:
import numpy as np
import bokeh.plotting as blt
import bokeh
bokeh.io.output_notebook()
import Cython
%load_ext Cython
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [2]:
import causticTools.sim as cSim
import causticTools.io as cIo
import causticTools.analytic as cAnalytic
import pyximport
pyximport.install()
from causticTools.cythonSim import massFunction

In [ ]:
M=1e9
m=9.99e8
rho=1
vFactor=(4/3)*np.pi
rho=1
class baryonInit(massFunction):
    def evaluate(self,r):
        return M
class baryonMass(massFunction):
    def evaluate(self,r):
        return m
class dmMass(massFunction):
    def evaluate(self,r):
        return vFactor*rho*(r**3)
class findEcc(massFunction):
    def evaluate(self,P):
        return 1-(1-P)**(1/n)

In [4]:
#test cell
nShells=64
nPhase=1
nEcc=1
T=1e7
dt=1e5
rMin=50
rMax=500
n=2 # defines eccentricity distribution
name='Kepler'+str(n)+'.r'+str(nShells)+'p'+str(2*nPhase)+'e'+str(nEcc)
nOutput=8
cSim.runSim(nShells,nPhase,nEcc,T,dt,rMin,rMax,name,nOutput,dmMass(),baryonInit(),baryonMass(),findEcc())

(130, ' edges')
(128, ' shells')
('Saving to file: ', 'output/Kepler.r64p2e1.0_4.txt')
('Saving to file: ', 'output/Kepler.r64p2e1.1_4.txt')
('Saving to file: ', 'output/Kepler.r64p2e1.2_4.txt')
('Saving to file: ', 'output/Kepler.r64p2e1.3_4.txt')
('Saving to file: ', 'output/Kepler.r64p2e1.4_4.txt')


In [ ]:
#production run cell
a=0
b=0
c=0
while (a<7):
    while (b<7):
        while (c<6):
            n=2**a # defines eccentricity distribution
            nShells=64
            nEcc=2**b
            nPhase=2**c
            T=1e7
            dt=1e5
            rMin=50
            rMax=500
            name='Kepler'+str(n)+'.r'+str(nShells)+'p'+str(2*nPhase)+'e'+str(nEcc)
            nOutput=8
            cSim.runSim(nShells,nPhase,nEcc,T,dt,rMin,rMax,name,nOutput,dmMass(),baryonInit(),baryonMass(),findEcc())
            c+=1
        b+=1
        c=0
    a+=1
    b=0
    c=0

(4160, ' edges')
(4096, ' shells')
('Saving to file: ', 'output/Kepler1.r64p2e32.0_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.1_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.2_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.3_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.4_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.5_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.6_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.7_8.txt')
('Saving to file: ', 'output/Kepler1.r64p2e32.8_8.txt')
(8320, ' edges')
(8192, ' shells')
('Saving to file: ', 'output/Kepler1.r64p4e32.0_8.txt')
('Saving to file: ', 'output/Kepler1.r64p4e32.1_8.txt')
('Saving to file: ', 'output/Kepler1.r64p4e32.2_8.txt')
('Saving to file: ', 'output/Kepler1.r64p4e32.3_8.txt')
('Saving to file: ', 'output/Kepler1.r64p4e32.4_8.txt')
('Saving to file: ', 'output/Kepler1.r64p4e32.5_8.txt')
('Saving to file: ', 'output/Kepler1.r64p4e32.6_8.txt')
('Saving to file: ', 'output/Keple

Exception ignored in: 'causticTools.cythonSim.findMass'
Traceback (most recent call last):
  File "causticTools/cythonSim.pyx", line 16, in causticTools.cythonSim.massFunction.evaluate (/Users/Zephyr/.pyxbld/temp.macosx-10.6-x86_64-3.5/pyrex/causticTools/cythonSim.c:2275)
    cpdef double evaluate(self,double r) except *:
  File "<ipython-input-3-08e25247b34b>", line 10, in evaluate
KeyboardInterrupt


('Saving to file: ', 'output/Kepler2.r64p32e32.7_8.txt')


In [ ]:
# production runs for normalisation (setting m=0.999M)
a=1
while (a<7):
    n=2**a # defines eccentricity distribution
    nShells=64
    nEcc=32
    nPhase=32
    T=1e7
    dt=1e5
    rMin=50
    rMax=500
    name='Normal'+str(n)
    nOutput=8
    cSim.runSim(nShells,nPhase,nEcc,T,dt,rMin,rMax,name,nOutput,dmMass(),baryonInit(),baryonMass(),findEcc())
    a+=1

(133120, ' edges')
(131072, ' shells')
('Saving to file: ', 'output/Normal1.0_8.txt')
('Saving to file: ', 'output/Normal1.1_8.txt')
('Saving to file: ', 'output/Normal1.2_8.txt')
('Saving to file: ', 'output/Normal1.3_8.txt')
('Saving to file: ', 'output/Normal1.4_8.txt')
('Saving to file: ', 'output/Normal1.5_8.txt')
('Saving to file: ', 'output/Normal1.6_8.txt')
('Saving to file: ', 'output/Normal1.7_8.txt')
('Saving to file: ', 'output/Normal1.8_8.txt')
(133120, ' edges')
(131072, ' shells')
('Saving to file: ', 'output/Normal2.0_8.txt')
('Saving to file: ', 'output/Normal2.1_8.txt')


Exception ignored in: 'causticTools.cythonSim.findMass'
Traceback (most recent call last):
  File "causticTools/cythonSim.pyx", line 16, in causticTools.cythonSim.massFunction.evaluate (/Users/Zephyr/.pyxbld/temp.macosx-10.6-x86_64-3.5/pyrex/causticTools/cythonSim.c:2275)
    cpdef double evaluate(self,double r) except *:
  File "<ipython-input-3-6b3e11efe890>", line 10, in evaluate
KeyboardInterrupt

In [45]:
rMin=50
rMax=500
dR=rMax-rMin
nOutput=4
figs=[]
for j in range(1,8):
    fig=blt.figure(x_range=(rMin-0.1*dR,rMax+0.1*dR),y_range=(0,5),width=500,height=250)
    for i in range(nOutput):
        fig=cIo.plotSmoothDens(fig,i,nOutput,'Kepler.r64p'+str(2**j)+'e1',1,rMin,rMax)
    figs.append(fig)
print(figs)
grid=bokeh.layouts.gridplot(figs,ncols=1)
blt.show(grid)

reading from:  output/Kepler.r64p2e1.0_4.txt
reading from:  output/Kepler.r64p2e1.1_4.txt
reading from:  output/Kepler.r64p2e1.2_4.txt
reading from:  output/Kepler.r64p2e1.3_4.txt
reading from:  output/Kepler.r64p4e1.0_4.txt
reading from:  output/Kepler.r64p4e1.1_4.txt
reading from:  output/Kepler.r64p4e1.2_4.txt
reading from:  output/Kepler.r64p4e1.3_4.txt
reading from:  output/Kepler.r64p8e1.0_4.txt
reading from:  output/Kepler.r64p8e1.1_4.txt
reading from:  output/Kepler.r64p8e1.2_4.txt
reading from:  output/Kepler.r64p8e1.3_4.txt
reading from:  output/Kepler.r64p16e1.0_4.txt
reading from:  output/Kepler.r64p16e1.1_4.txt
reading from:  output/Kepler.r64p16e1.2_4.txt
reading from:  output/Kepler.r64p16e1.3_4.txt
reading from:  output/Kepler.r64p32e1.0_4.txt
reading from:  output/Kepler.r64p32e1.1_4.txt
reading from:  output/Kepler.r64p32e1.2_4.txt
reading from:  output/Kepler.r64p32e1.3_4.txt
reading from:  output/Kepler.r64p64e1.0_4.txt
reading from:  output/Kepler.r64p64e1.1_4.txt


In [18]:
rMin=50
rMax=500
dR=rMax-rMin
nOutput=4
fig=blt.figure(x_range=(rMin-0.1*dR,rMax+0.1*dR),y_range=(0,5))
for i in range(nOutput):
    fig=cIo.plotSmoothDens(fig,i,nOutput,'Kepler.r32p64e16',1,rMin,rMax)
blt.show(fig)

reading from:  output/Kepler.r32p64e16.0_4.txt
reading from:  output/Kepler.r32p64e16.1_4.txt
reading from:  output/Kepler.r32p64e16.2_4.txt
reading from:  output/Kepler.r32p64e16.3_4.txt


In [9]:
aR,aDens=cAnalytic.densProfile(rMin,rMax,128,M,m,7.5e6)

/Users/Zephyr/Google Drive/astro/caustics/causticTools/analytic.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return ((G*m*(t**2))*((1-eps*np.cos(eta))**3)/((eta-eps*np.sin(eta))**2))**(1/3)


In [10]:
dR=rMax-rMin
fig=blt.figure(x_range=(rMin-0.1*dR,rMax+0.1*dR),y_range=(0,5))
fig=cIo.plotDensProfile(fig,aR,aDens,rMin,rMax,0.5)
fig=cIo.plotDens(fig,3,nOutput,'1phase')
fig=cIo.plotSmoothDens(fig,3,nOutput,'1phase',1,rMin,rMax)
blt.show(fig)

In [6]:
%%cython
cdef class Function:
    cpdef double evaluate(self, double x) except *:
        return 0

cdef Function globalFunction
cdef Function globalFunction2
    
cpdef setFunction(Function f, Function f2):
    global globalFunction, globalFunction2
    globalFunction=f
    globalFunction2=f2

cpdef printFunction(double r):
    print(globalFunction2.evaluate(r))

def integrate(Function f, double a, double b, int N):
    cdef int i
    cdef double s, dx
    if f is None:
        raise ValueError("f cannot be None")
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f.evaluate(a+i*dx)
    return s * dx

In [7]:
class MyPolynomial():
    def evaluate(self, x):
        return 2*x*x + 3*x - 12
    
class MyPolynomial2():
    def evaluate(self, x):
        return 2*x*x + 3*x - 6

integrate(MyPolynomial(), 0, 1, 10000)

setFunction(MyPolynomial(),MyPolynomial2())
printFunction(1)

TypeError: Argument 'f' has incorrect type (expected _cython_magic_b914fad814b3f57d6899ec1f1b6a9fa7.Function, got MyPolynomial)